In [1]:
import os
import pandas as pd

In [2]:
os.chdir('c:\\Users\\tyler\\OneDrive\\Documents\\Python\\NFL')
os.getcwd()

'c:\\Users\\tyler\\OneDrive\\Documents\\Python\\NFL'

In [3]:
files = list(set(os.listdir('backend/data/odds')).difference({'odds.csv', 'odds.ipynb', 'nfl odds 2022-23.xlsx', 'nfl odds 2021-22.xlsx'}))
files

['nfl odds 2017-18.xlsx',
 'nfl odds 2011-12.xlsx',
 'nfl odds 2016-17.xlsx',
 'nfl odds 2010-11.xlsx',
 'nfl odds 2019-20.xlsx',
 'nfl odds 2012-13.xlsx',
 'nfl odds 2018-19.xlsx',
 'nfl odds 2015-16.xlsx',
 'nfl odds 2014-15.xlsx',
 'nfl odds 2013-14.xlsx',
 'nfl odds 2020-21.xlsx']

In [4]:
def spreads(col1, col2):
    if col1 == "pk":
        col1 = 0
    if col2 == "pk":
        col2 = 0
    if col1 < col2:
        return col1
    else:
        return col2

In [5]:
def totals(col1, col2):
    if col1 == "pk":
        col1 = 0
    if col2 == "pk":
        col2 = 0
    if col1 < col2:
        return col2
    else:
        return col1

In [6]:
def odds_season(filename):
    df = pd.read_excel(f'backend/data/odds/{filename}')
    df.columns = [col.lower() for col in df.columns]

    df["year"] = filename.split("-")[0][-4:]

    home_df = df[df["vh"] == "H"].reset_index()[["team", "ml", "close", "year", "week"]]
    home_df.rename(columns={"team": "home"}, inplace=True)

    away_df = df[df["vh"] == "V"].reset_index()[["team", "ml", "close", "year", "week"]]
    away_df.rename(columns={"team": "away"}, inplace=True)

    df = home_df.join(away_df, lsuffix="_h", rsuffix="_a")

    df['spread'] = df.apply(lambda x: spreads(x.close_h, x.close_a), axis=1)
    df['total'] = df.apply(lambda x: totals(x.close_h, x.close_a), axis=1)
    df.drop(["close_h", "close_a"], axis=1, inplace=True)

    df.drop(["week_a", "year_a"], axis=1, inplace=True)
    df.rename(columns={"year_h": "year", "week_h": "week"}, inplace=True)

    teams_dict = {
        'Philadelphia': 'philadelphia-eagles', 'St.Louis': 'st-louis-rams',
        'TampaBay': 'tampa-bay-buccaneers', 'NYGiants': 'new-york-giants', 'GreenBay': 'green-bay-packers',
        'Chicago': 'chicago-bears', 'NewEngland': 'new-england-patriots', 'Pittsburgh': 'pittsburgh-steelers',
        'Houston': 'houston-texans', 'Denver': 'denver-broncos', 'SanFrancisco': 'san-francisco-49ers',
        'Minnesota': 'minnesota-vikings', 'Washington': 'washington-redskins',
        'Jacksonville': 'jacksonville-jaguars', 'Tennessee': 'tennessee-titans',
        'Carolina': 'carolina-panthers', 'KansasCity': 'kansas-city-chiefs', 'Miami': 'miami-dolphins',
        'Atlanta': 'atlanta-falcons', 'NewOrleans': 'new-orleans-saints', 'Baltimore': 'baltimore-ravens',
        'Seattle': 'seattle-seahawks', 'SanDiego': 'san-diego-chargers', 'Dallas': 'dallas-cowboys',
        'Cincinnati': 'cincinnati-bengals', 'NYJets': 'new-york-jets', 'Detroit': 'detroit-lions',
        'Arizona': 'arizona-cardinals', 'Oakland': 'oakland-raiders', 'Indianapolis': 'indianapolis-colts',
        'Buffalo': 'buffalo-bills', 'Cleveland': 'cleveland-browns', 'LARams': 'los-angeles-rams',
        'LAChargers': 'los-angeles-chargers', 'washington': 'washington-football-team',
        'LasVegas': 'las-vegas-raiders'
    }

    df["home"] = df["home"].apply(lambda x: teams_dict[x])
    df["away"] = df["away"].apply(lambda x: teams_dict[x])

    return df

In [7]:
odds = pd.DataFrame()
for file in files:
    odds = odds.append(odds_season(file))
odds = pd.concat(
    [
        odds, 
        pd.read_excel('backend\\data\\odds\\nfl odds 2021-22.xlsx'),
        pd.read_excel('backend\\data\\odds\\nfl odds 2022-23.xlsx')
    ],
    axis=0
)
odds["home"] = odds["home"].apply(lambda x: ' '.join([word.capitalize() for word in x.split('-')]))
odds["away"] = odds["away"].apply(lambda x: ' '.join([word.capitalize() for word in x.split('-')]))
odds.rename({'year': 'season'}, axis=1, inplace=True)

odds.tail(20)

,home,ml_h,season,week,away,ml_a,spread,total
60,Las Vegas Raiders,-145,2022,4,Denver Broncos,125,2.5,45.5
61,Green Bay Packers,-425,2022,4,New England Patriots,370,9.5,40.0
62,Tampa Bay Buccaneers,-110,2022,4,Kansas City Chiefs,-110,1.0,45.5
63,San Francisco 49ers,-125,2022,4,Los Angeles Rams,110,1.5,42.5
64,Denver Broncos,-170,2022,5,Indianapolis Colts,150,3.0,42.5
65,Green Bay Packers,-360,2022,5,New York Giants,300,9.0,41.5
66,New Orleans Saints,-220,2022,5,Seattle Seahawks,185,5.0,45.5
67,Jacksonville Jaguars,-315,2022,5,Houston Texans,260,7.0,43.5
68,Buffalo Bills,-830,2022,5,Pittsburgh Steelers,700,14.0,46.5
69,Tampa Bay Buccaneers,-460,2022,5,Atlanta Falcons,380,10.0,46.5


In [8]:
odds.to_csv('backend\\data\\odds\\odds.csv', index=False)